In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier, Pool
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors, MolSurf, AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from scipy import stats
import pickle
%matplotlib inline

In [ ]:
# Загружаем файл Excel
df = pd.read_csv('data/SEC_SYN_with_features.csv')

# Устанавливаем опцию для отображения всех колонок (если это необходимо для отладки, иначе можно убрать)
pd.set_option('display.max_columns', None)

### Стратифицированное разбиение по металлу

In [ ]:
print(df["Металл"].value_counts())

features_for_metal_prediction = [
    # Исходные данные СЭХ
    'W0, см3/г', 
    'E0, кДж/моль',
    'х0, нм', 
    'а0, ммоль/г', 
    'E, кДж/моль', 
    'SБЭТ, м2/г', 
    'Ws, см3/г', 
    'Sme, м2/г', 
    'Wme, см3/г',
    
    # Признаки, сгенерированные из данных СЭХ
    'Adsorption_Potential', 
    'Capacity_Density', 
    'K_equilibrium', 
    'Delta_G',
    'SurfaceArea_MicroVol_Ratio',
    'Adsorption_Energy_Ratio', 
    'S_BET_E', 
    'x0_W0',
    'B_micropore'
]

In [ ]:
# ===================================================================
# 1. Импорт библиотек
# ===================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib # Для сохранения модели

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Игнорируем предупреждения для чистоты вывода
import warnings
warnings.filterwarnings('ignore')

# ===================================================================
# 2. Подготовка данных
# ===================================================================

# Определяем признаки (X) и целевую переменную (y)
features = features_for_metal_prediction
target = 'Металл'

# Удаляем строки с пропущенными значениями в наших признаках или цели
data_for_model = df[features + [target]].dropna()

X = data_for_model[features]
y = data_for_model[target]

# Кодируем текстовые метки (названия металлов) в числа (0, 1, 2...)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Разделяем данные на обучающую и тестовую выборки (75% / 25%)
# stratify=y_encoded гарантирует, что пропорции классов в обеих выборках будут одинаковыми
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.25, 
    random_state=42, # для воспроизводимости результатов
    stratify=y_encoded
)

print("Данные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")
print(f"Классы металлов: {label_encoder.classes_}")

# ===================================================================
# 3. Обучение модели "Случайный Лес"
# ===================================================================

# Создаем модель RandomForestClassifier
# n_estimators - количество "деревьев" в лесу. Больше - лучше, но дольше.
# class_weight='balanced' - автоматически взвешивает классы, чтобы компенсировать дисбаланс.
# n_jobs=-1 - использовать все доступные ядра процессора для ускорения обучения.
model = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    oob_score=True # Использовать out-of-bag оценку для доп. валидации
)

print("\nНачинаем обучение модели 'Случайный Лес'...")
model.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных (внутренняя оценка): {model.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели на тестовых данных
# ===================================================================

# Делаем предсказания на тестовой выборке, которую модель никогда не видела
y_pred = model.predict(X_test)

# Оцениваем качество
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность на тестовой выборке: {accuracy:.4f}")

# Детальный отчет по каждому классу (точность, полнота, F1-мера)
print("\nОтчет по классификации:")
# Преобразуем числовые метки обратно в названия металлов для наглядности
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================

# --- Матрица ошибок (Confusion Matrix) ---
# Показывает, какие классы модель путает между собой
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Матрица ошибок', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()


# --- Важность признаков (Feature Importance) ---
# Показывает, какие признаки внесли наибольший вклад в предсказание
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 10))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df, palette='viridis')
plt.title('Важность признаков для предсказания металла', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))


# ===================================================================
# 6. Сохранение модели и кодировщика
# ===================================================================

# Сохраняем обученную модель и кодировщик для будущего использования
joblib.dump(model, 'metal_classifier_model.pkl')
joblib.dump(label_encoder, 'metal_label_encoder.pkl')

print("\nМодель ('metal_classifier_model.pkl') и кодировщик ('metal_label_encoder.pkl') успешно сохранены.")



In [ ]:
import joblib

# Укажите правильный путь к вашему файлу
MODEL_FILE = 'metal_classifier_model.pkl'

try:
    # Загружаем объект из файла
    loaded_object = joblib.load(MODEL_FILE)

    # Проверяем тип загруженного объекта
    print(f"Файл '{MODEL_FILE}' содержит объект типа: {type(loaded_object)}")

    # Если это модель scikit-learn, у нее должен быть метод .predict
    # Этот код вызовет ошибку, если это не модель
    if hasattr(loaded_object, 'predict'):
        print("✅ Это корректный объект модели.")
    else:
        print("❌ ОШИБКА: Это НЕ объект модели. Похоже, это массив NumPy или что-то другое.")

except FileNotFoundError:
    print(f"Ошибка: Файл '{MODEL_FILE}' не найден.")
except Exception as e:
    print(f"Произошла ошибка при загрузке: {e}")

### Стратифицированное разбиение по Лиганду

In [ ]:
df["Лиганд"].value_counts()

### Обучаем вторую модельку с использованием дескрипторов металла, предсказываем лиганд

In [ ]:
# ===================================================================
# 1. Импорт библиотек
# ===================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Определяем базовые группы признаков ---
# 1. Свойства адсорбента (СЭХ и производные)
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г',
    'Adsorption_Potential', 'Capacity_Density', 'K_equilibrium', 'Delta_G',
    'SurfaceArea_MicroVol_Ratio', 'Adsorption_Energy_Ratio', 'S_BET_E', 
    'x0_W0', 'B_micropore'
]
# 2. Дескрипторы металла
features_metal_descriptors = [
    'Total molecular weight (metal)',
    'Average ionic radius (metal)', 
    'Average electronegativity (metal)'
]

# --- Применяем One-Hot Encoding к категориальной переменной 'Металл' ---
df_encoded = pd.get_dummies(df, columns=['Металл'], prefix='Металл', dtype=int)

# Получаем список новых столбцов, созданных OHE
ohe_metal_columns = [col for col in df_encoded.columns if col.startswith('Металл_')]

# --- Собираем финальный список признаков ---
final_features_for_ligand = features_seh + features_metal_descriptors + ohe_metal_columns
target_ligand = 'Лиганд'

# --- Подготовка данных для модели ---
# Удаляем строки с пропущенными значениями
data_for_model = df_encoded[final_features_for_ligand + [target_ligand]].dropna()

X = data_for_model[final_features_for_ligand]
y = data_for_model[target_ligand]

# Кодируем текстовые метки лигандов в числа
label_encoder_ligand = LabelEncoder()
y_encoded = label_encoder_ligand.fit_transform(y)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.25, 
    random_state=42,
    stratify=y_encoded # Важно для сохранения пропорций редких классов
)

print("Данные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")
print(f"Классы лигандов: {label_encoder_ligand.classes_}")

# ===================================================================
# 3. Обучение модели "Случайный Лес"
# ===================================================================

# Создаем модель с автоматической балансировкой классов
model_ligand = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    oob_score=True
)

print("\nНачинаем обучение модели для предсказания лиганда...")
model_ligand.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных (внутренняя оценка): {model_ligand.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели на тестовых данных
# ===================================================================

y_pred = model_ligand.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность на тестовой выборке: {accuracy:.4f}")

print("\nОтчет по классификации:")
report = classification_report(y_test, y_pred, target_names=label_encoder_ligand.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================

# --- Матрица ошибок ---
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=label_encoder_ligand.classes_, yticklabels=label_encoder_ligand.classes_)
plt.title('Матрица ошибок для предсказания лиганда', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()

# --- Важность признаков ---
importances = model_ligand.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features_for_ligand,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 12))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df, palette='mako')
plt.title('Важность признаков для предсказания лиганда', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение модели и кодировщика
# ===================================================================

joblib.dump(model_ligand, 'ligand_classifier_model.pkl')
joblib.dump(label_encoder_ligand, 'ligand_label_encoder.pkl')

print("\nМодель ('ligand_classifier_model.pkl') и кодировщик ('ligand_label_encoder.pkl') успешно сохранены.")

### Предсказание рас-ля

In [ ]:
df["Растворитель"].value_counts()

In [ ]:
# ===================================================================
# 1. Импорт библиотек
# ===================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Игнорируем предупреждения для чистоты вывода
import warnings
warnings.filterwarnings('ignore')

# --- КЛЮЧЕВОЙ ШАГ: Группировка классов растворителей ---
def group_solvent(solvent_name):
    if not isinstance(solvent_name, str):
        return 'Другой'
    if solvent_name == 'ДМФА':
        return 'ДМФА'
    elif 'ДМФА' in solvent_name:
        return 'Смесь на основе ДМФА'
    else:
        return 'Другой'

df['Растворитель_группа'] = df['Растворитель'].apply(group_solvent)
print("Распределение растворителей после группировки:")
print(df['Растворитель_группа'].value_counts())

# --- Определяем группы признаков ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г',
    'Adsorption_Potential', 'Capacity_Density', 'K_equilibrium', 'Delta_G',
    'SurfaceArea_MicroVol_Ratio', 'Adsorption_Energy_Ratio', 'S_BET_E', 
    'x0_W0', 'B_micropore'
]
features_metal_descriptors = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)'
]
features_ligand_descriptors = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)'
]

# --- Применяем One-Hot Encoding к металлу и лиганду ---
df_encoded = pd.get_dummies(df, columns=['Металл', 'Лиганд'], prefix=['Металл', 'Лиганд'], dtype=int)

ohe_metal_columns = [col for col in df_encoded.columns if col.startswith('Металл_')]
ohe_ligand_columns = [col for col in df_encoded.columns if col.startswith('Лиганд_')]

# --- Собираем финальный список признаков ---
final_features_for_solvent = (
    features_seh + 
    features_metal_descriptors + ohe_metal_columns +
    features_ligand_descriptors + ohe_ligand_columns
)
target_solvent = 'Растворитель_группа' # Наша новая, сгруппированная цель

# --- Подготовка данных для модели ---
data_for_model = df_encoded[final_features_for_solvent + [target_solvent]].dropna()

X = data_for_model[final_features_for_solvent]
y = data_for_model[target_solvent]

label_encoder_solvent = LabelEncoder()
y_encoded = label_encoder_solvent.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

print("\nДанные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")
print(f"Классы растворителей: {label_encoder_solvent.classes_}")

# ===================================================================
# 3. Обучение модели "Случайный Лес"
# ===================================================================
model_solvent = RandomForestClassifier(
    n_estimators=200, class_weight='balanced', random_state=42, n_jobs=-1, oob_score=True
)

print("\nНачинаем обучение модели для предсказания группы растворителя...")
model_solvent.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных (внутренняя оценка): {model_solvent.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_solvent.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность на тестовой выборке: {accuracy:.4f}")

print("\nОтчет по классификации:")
report = classification_report(y_test, y_pred, target_names=label_encoder_solvent.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=label_encoder_solvent.classes_, yticklabels=label_encoder_solvent.classes_)
plt.title('Матрица ошибок для предсказания группы растворителя', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()

importances = model_solvent.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features_for_solvent,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='plasma')
plt.title('Важность признаков для предсказания группы растворителя', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение модели и кодировщика
# ===================================================================
joblib.dump(model_solvent, 'solvent_classifier_model.pkl')
joblib.dump(label_encoder_solvent, 'solvent_label_encoder.pkl')

print("\nМодель ('solvent_classifier_model.pkl') и кодировщик ('solvent_label_encoder.pkl') успешно сохранены.")


#### Предсказание напрямую массы соли

In [ ]:
# ===================================================================
# 1. Импорт библиотек
# ===================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from pyod.models.iforest import IForest

# Игнорируем предупреждения для чистоты вывода
import warnings
warnings.filterwarnings('ignore')

# --- Шаг 1: Удаление выбросов в целевой переменной ---
print(f"Исходное количество строк: {len(df)}")
iso_forest = IForest(contamination=0.05, random_state=42)
df_for_iso = df[['m (соли), г']].dropna()
outliers = iso_forest.fit_predict(df_for_iso)
clean_indices = df_for_iso.index[outliers == 0]
df_cleaned = df.loc[clean_indices].copy()
print(f"Количество строк после удаления выбросов: {len(df_cleaned)}")

# --- Шаг 2: ЯВНОЕ ОПРЕДЕЛЕНИЕ КОРРЕКТНЫХ ПРИЗНАКОВ ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г',
    'Adsorption_Potential', 'Capacity_Density', 'K_equilibrium', 'Delta_G',
    'SurfaceArea_MicroVol_Ratio', 'Adsorption_Energy_Ratio', 'S_BET_E', 
    'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)',
    'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']
target = 'm (соли), г'

# Собираем все нужные нам столбцы в один список
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + categorical_features + [target]
)
# Убираем дубликаты и проверяем наличие колонок
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df_cleaned.columns)))

# Создаем изолированный DataFrame только с нужными данными
df_isolated = df_cleaned[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y = df_encoded[target]

# Сохраняем финальный список признаков для графика важности
final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("\nДанные успешно подготовлены с корректным набором признаков:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели регрессии "Случайный Лес"
# ===================================================================
model_mass_regressor = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, oob_score=True
)

print("\nНачинаем обучение регрессионной модели...")
model_mass_regressor.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Коэффициент детерминации R² на OOB данных: {model_mass_regressor.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_mass_regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nОценка на тестовой выборке:")
print(f"Коэффициент детерминации (R²): {r2:.4f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.4f} г")

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
plt.title(f'Предсказание массы соли (R² = {r2:.4f})', fontsize=16)
plt.xlabel('Реальная масса (г)', fontsize=12)
plt.ylabel('Предсказанная масса (г)', fontsize=12)
plt.grid(True)
plt.show()

importances = model_mass_regressor.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='cubehelix')
plt.title('Важность признаков для предсказания массы соли', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение модели
# ===================================================================
joblib.dump(model_mass_regressor, 'mass_regressor_model.pkl')
print("\nМодель регрессии ('mass_regressor_model.pkl') успешно сохранена.")


#### Предсказание массы кислоты

In [ ]:
# --- Новая целевая переменная ---
target = 'm(кис-ты), г'

# --- Шаг 1: Удаление выбросов в новой целевой переменной ---
print(f"Исходное количество строк: {len(df)}")
iso_forest = IForest(contamination=0.05, random_state=42)
df_for_iso = df[[target]].dropna()
outliers = iso_forest.fit_predict(df_for_iso)
clean_indices = df_for_iso.index[outliers == 0]
df_cleaned = df.loc[clean_indices].copy()
print(f"Количество строк после удаления выбросов: {len(df_cleaned)}")

# --- Шаг 2: Определяем корректный набор признаков ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г', 'Adsorption_Potential', 
    'Capacity_Density', 'K_equilibrium', 'Delta_G', 'SurfaceArea_MicroVol_Ratio', 
    'Adsorption_Energy_Ratio', 'S_BET_E', 'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)', 'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
# --- ДОБАВЛЯЕМ ПРИЗНАКИ ИЗ ПРЕДЫДУЩЕГО ШАГА ---
features_from_prev_step = [
    'm (соли), г',
    'n_соли'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']

# Собираем все нужные нам столбцы в один список
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + features_from_prev_step + 
    categorical_features + [target]
)
# Убираем дубликаты и проверяем наличие колонок
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df_cleaned.columns)))
df_isolated = df_cleaned[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y = df_encoded[target]
final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("\nДанные для предсказания массы кислоты успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели регрессии "Случайный Лес"
# ===================================================================
model_acid_regressor = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, oob_score=True
)

print("\nНачинаем обучение регрессионной модели для массы кислоты...")
model_acid_regressor.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Коэффициент детерминации R² на OOB данных: {model_acid_regressor.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_acid_regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nОценка на тестовой выборке:")
print(f"Коэффициент детерминации (R²): {r2:.4f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.4f} г")

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
plt.title(f'Предсказание массы кислоты (R² = {r2:.4f})', fontsize=16)
plt.xlabel('Реальная масса (г)', fontsize=12)
plt.ylabel('Предсказанная масса (г)', fontsize=12)
plt.grid(True)
plt.show()

importances = model_acid_regressor.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='viridis')
plt.title('Важность признаков для предсказания массы кислоты', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение модели
# ===================================================================
joblib.dump(model_acid_regressor, 'acid_mass_regressor_model.pkl')
print("\nМодель регрессии для массы кислоты ('acid_mass_regressor_model.pkl') успешно сохранена.")

### Предсказание объема рас-ля

In [ ]:
# --- Новая целевая переменная ---
target = 'Vсин. (р-ля), мл'

# --- Шаг 1: Удаление выбросов в новой целевой переменной ---
print(f"Исходное количество строк: {len(df)}")
iso_forest = IForest(contamination=0.05, random_state=42)
df_for_iso = df[[target]].dropna()
outliers = iso_forest.fit_predict(df_for_iso)
clean_indices = df_for_iso.index[outliers == 0]
df_cleaned = df.loc[clean_indices].copy()
print(f"Количество строк после удаления выбросов: {len(df_cleaned)}")

# --- Шаг 2: Определяем корректный набор признаков ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г', 'Adsorption_Potential', 
    'Capacity_Density', 'K_equilibrium', 'Delta_G', 'SurfaceArea_MicroVol_Ratio', 
    'Adsorption_Energy_Ratio', 'S_BET_E', 'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)', 'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
features_from_prev_steps = [
    'm (соли), г', 'n_соли', 'm(кис-ты), г', 'n_кислоты'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']

# Собираем все нужные нам столбцы в один список
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + features_from_prev_steps + 
    categorical_features + [target]
)
# Убираем дубликаты и проверяем наличие колонок
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df_cleaned.columns)))
df_isolated = df_cleaned[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y = df_encoded[target]
final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("\nДанные для предсказания объема растворителя успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели регрессии "Случайный Лес"
# ===================================================================
model_volume_regressor = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1, oob_score=True
)

print("\nНачинаем обучение регрессионной модели для объема растворителя...")
model_volume_regressor.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Коэффициент детерминации R² на OOB данных: {model_volume_regressor.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_volume_regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nОценка на тестовой выборке:")
print(f"Коэффициент детерминации (R²): {r2:.4f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.4f} мл")

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
plt.figure(figsize=(8, 8))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
plt.title(f'Предсказание объема растворителя (R² = {r2:.4f})', fontsize=16)
plt.xlabel('Реальный объем (мл)', fontsize=12)
plt.ylabel('Предсказанный объем (мл)', fontsize=12)
plt.grid(True)
plt.show()

importances = model_volume_regressor.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='magma')
plt.title('Важность признаков для предсказания объема растворителя', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение модели
# ===================================================================
joblib.dump(model_volume_regressor, 'volume_regressor_model.pkl')
print("\nМодель регрессии для объема растворителя ('volume_regressor_model.pkl') успешно сохранена.")

### Т.син., °С подведем под задачу классификации (сначала классификацию рассмотрим)

In [ ]:
# ===================================================================
# 1. Импорт библиотек
# ===================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import randint, uniform


# --- Шаг 1: Создание категорий для температуры ---
# Используем pd.cut для создания осмысленных температурных диапазонов
# Учитывая ваше распределение, выделим 3 группы
bins = [0, 115, 135, 250]
labels = ['Низкая (<115°C)', 'Средняя (115-135°C)', 'Высокая (>135°C)']
df['Tsyn_Category'] = pd.cut(df['Т.син., °С'], bins=bins, labels=labels, right=False)
print("Распределение температурных категорий:")
print(df['Tsyn_Category'].value_counts())

# --- Шаг 2: Определение признаков и цели ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г', 'Adsorption_Potential', 
    'Capacity_Density', 'K_equilibrium', 'Delta_G', 'SurfaceArea_MicroVol_Ratio', 
    'Adsorption_Energy_Ratio', 'S_BET_E', 'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)', 'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
features_from_prev_steps = [
    'm (соли), г', 'n_соли', 'm(кис-ты), г', 'n_кислоты', 'Vсин. (р-ля), мл'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']
target = 'Tsyn_Category' # Наша новая категориальная цель

# Собираем все нужные нам столбцы
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + features_from_prev_steps + 
    categorical_features + [target]
)
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df.columns)))
df_isolated = df[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y_text = df_encoded[target] # Текстовые метки

# Кодируем текстовые метки в числа
label_encoder_temp = LabelEncoder()
y = label_encoder_temp.fit_transform(y_text)

final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("\nДанные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели классификации
# ===================================================================
model_temp_classifier = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced', # Важно для несбалансированных классов
    random_state=42,
    n_jobs=-1,
    oob_score=True
)

print("\nНачинаем обучение модели классификации для температуры...")
model_temp_classifier.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных: {model_temp_classifier.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_temp_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nОценка на тестовой выборке:")
print(f"Точность: {accuracy:.4f}")

print("\nОтчет по классификации:")
report = classification_report(y_test, y_pred, target_names=label_encoder_temp.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='BuPu', 
            xticklabels=label_encoder_temp.classes_, yticklabels=label_encoder_temp.classes_)
plt.title('Матрица ошибок для предсказания категории температуры', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()

importances = model_temp_classifier.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='rocket')
plt.title('Важность признаков для предсказания категории температуры', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение финальной модели
# ===================================================================
joblib.dump(model_temp_classifier, 'temperature_classifier_model.pkl')
joblib.dump(label_encoder_temp, 'temperature_label_encoder.pkl')
print("\nМодель классификации ('temperature_classifier_model.pkl') и кодировщик ('temperature_label_encoder.pkl') успешно сохранены.")

### Т суш., °С подведем под задачу классификации

In [ ]:
# --- Шаг 1: Создание категорий для температуры сушки ---
bins = [0, 115, 135, 250]
labels = ['Низкая (<115°C)', 'Средняя (115-135°C)', 'Высокая (>135°C)']
df['Tdry_Category'] = pd.cut(df['Т суш., °С'], bins=bins, labels=labels, right=False)
print("Распределение категорий температуры сушки:")
print(df['Tdry_Category'].value_counts())

# --- Шаг 2: Определение признаков и цели ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г', 'Adsorption_Potential', 
    'Capacity_Density', 'K_equilibrium', 'Delta_G', 'SurfaceArea_MicroVol_Ratio', 
    'Adsorption_Energy_Ratio', 'S_BET_E', 'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)', 'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
features_from_prev_steps = [
    'm (соли), г', 'n_соли', 'm(кис-ты), г', 'n_кислоты', 'Vсин. (р-ля), мл', 'Т.син., °С'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']
target = 'Tdry_Category' # Наша новая категориальная цель

# Собираем все нужные нам столбцы
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + features_from_prev_steps + 
    categorical_features + [target]
)
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df.columns)))
df_isolated = df[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y_text = df_encoded[target]

# Кодируем текстовые метки в числа
label_encoder_tdry = LabelEncoder()
y = label_encoder_tdry.fit_transform(y_text)

final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("\nДанные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели классификации
# ===================================================================
model_tdry_classifier = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    oob_score=True
)

print("\nНачинаем обучение модели классификации для температуры сушки...")
model_tdry_classifier.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных: {model_tdry_classifier.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_tdry_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nОценка на тестовой выборке:")
print(f"Точность: {accuracy:.4f}")

print("\nОтчет по классификации:")
report = classification_report(y_test, y_pred, target_names=label_encoder_tdry.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=label_encoder_tdry.classes_, yticklabels=label_encoder_tdry.classes_)
plt.title('Матрица ошибок для предсказания категории температуры сушки', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()

importances = model_tdry_classifier.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='viridis_r')
plt.title('Важность признаков для предсказания категории температуры сушки', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение финальной модели
# ===================================================================
joblib.dump(model_tdry_classifier, 'drying_temp_classifier_model.pkl')
joblib.dump(label_encoder_tdry, 'drying_temp_label_encoder.pkl')
print("\nМодель классификации ('drying_temp_classifier_model.pkl') и кодировщик ('drying_temp_label_encoder.pkl') успешно сохранены.")


### Температура регенерации как задача классификации

In [ ]:
# --- Шаг 1: Создание категорий для температуры регенерации ---
bins = [0, 155, 265, 500]
labels = ['Низкая (<155°C)', 'Средняя (155-265°C)', 'Высокая (>265°C)']
df['Treg_Category'] = pd.cut(df['Tрег, ᵒС'], bins=bins, labels=labels, right=False)
print("Распределение категорий температуры регенерации:")
print(df['Treg_Category'].value_counts())

# --- Шаг 2: Определение признаков и цели ---
features_seh = [
    'W0, см3/г', 'E0, кДж/моль', 'х0, нм', 'а0, ммоль/г', 'E, кДж/моль', 
    'SБЭТ, м2/г', 'Ws, см3/г', 'Sme, м2/г', 'Wme, см3/г', 'Adsorption_Potential', 
    'Capacity_Density', 'K_equilibrium', 'Delta_G', 'SurfaceArea_MicroVol_Ratio', 
    'Adsorption_Energy_Ratio', 'S_BET_E', 'x0_W0', 'B_micropore'
]
features_metal_all = [
    'Total molecular weight (metal)', 'Average ionic radius (metal)', 'Average electronegativity (metal)',
    'Молярка_соли'
]
features_ligand_all = [
    'carboxyl_groups (ligand)', 'aromatic_rings (ligand)', 'carbon_atoms (ligand)', 
    'oxygen_atoms (ligand)', 'nitrogen_atoms (ligand)', 'molecular_weight (ligand)',
    'amino_groups (ligand)', 'logP (ligand)', 'TPSA (ligand)', 
    'h_bond_acceptors (ligand)', 'h_bond_donors (ligand)', 'Молярка_кислоты'
]
features_solvent_all = [
    'Solvent_MolWt', 'Solvent_LogP', 'Solvent_NumHDonors', 'Solvent_NumHAcceptors'
]
features_from_prev_steps = [
    'm (соли), г', 'n_соли', 'm(кис-ты), г', 'n_кислоты', 'Vсин. (р-ля), мл', 'Т.син., °С', 'Т суш., °С'
]
categorical_features = ['Металл', 'Лиганд', 'Растворитель']
target = 'Treg_Category' # Наша новая категориальная цель

# Собираем все нужные нам столбцы
all_needed_columns = (
    features_seh + features_metal_all + features_ligand_all + 
    features_solvent_all + features_from_prev_steps + 
    categorical_features + [target]
)
all_needed_columns = sorted(list(set(col for col in all_needed_columns if col in df.columns)))
df_isolated = df[all_needed_columns].dropna()

# --- Шаг 3: One-Hot Encoding ---
df_encoded = pd.get_dummies(df_isolated, columns=categorical_features, prefix=categorical_features, dtype=int)

# --- Шаг 4: Подготовка финальных данных для модели ---
X = df_encoded.drop(columns=[target])
y_text = df_encoded[target]

# Кодируем текстовые метки в числа
label_encoder_treg = LabelEncoder()
y = label_encoder_treg.fit_transform(y_text)

final_features = X.columns.tolist()

# --- Шаг 5: Разделение данных ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("\nДанные успешно подготовлены:")
print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")
print(f"Количество признаков: {X_train.shape[1]}")

# ===================================================================
# 3. Обучение модели классификации
# ===================================================================
model_treg_classifier = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    oob_score=True
)

print("\nНачинаем обучение модели классификации для температуры регенерации...")
model_treg_classifier.fit(X_train, y_train)
print("Обучение завершено.")
print(f"Точность модели на OOB данных: {model_treg_classifier.oob_score_:.4f}")

# ===================================================================
# 4. Оценка производительности модели
# ===================================================================
y_pred = model_treg_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nОценка на тестовой выборке:")
print(f"Точность: {accuracy:.4f}")

print("\nОтчет по классификации:")
report = classification_report(y_test, y_pred, target_names=label_encoder_treg.classes_)
print(report)

# ===================================================================
# 5. Визуализация результатов
# ===================================================================
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='cividis', 
            xticklabels=label_encoder_treg.classes_, yticklabels=label_encoder_treg.classes_)
plt.title('Матрица ошибок для предсказания категории температуры регенерации', fontsize=16)
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.show()

importances = model_treg_classifier.feature_importances_
feature_importance_df = pd.DataFrame({
    'Признак': final_features,
    'Важность': importances
}).sort_values(by='Важность', ascending=False)

plt.figure(figsize=(12, 14))
sns.barplot(x='Важность', y='Признак', data=feature_importance_df.head(25), palette='inferno')
plt.title('Важность признаков для предсказания категории температуры регенерации', fontsize=16)
plt.xlabel('Важность', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.tight_layout()
plt.show()

print("\nТоп-10 самых важных признаков:")
print(feature_importance_df.head(10))

# ===================================================================
# 6. Сохранение финальной модели
# ===================================================================
joblib.dump(model_treg_classifier, 'regen_temp_classifier_model.pkl')
joblib.dump(label_encoder_treg, 'regen_temp_label_encoder.pkl')
print("\nМодель классификации ('regen_temp_classifier_model.pkl') и кодировщик ('regen_temp_label_encoder.pkl') успешно сохранены.")